In [1]:
import cv2
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

from imutils import face_utils
font = cv2.FONT_HERSHEY_SIMPLEX

In [2]:
import os

def get_files(path):
    return os.listdir(path)

cascPath = "/Users/abdulrehman/opt/anaconda3/envs/Face-Detection/lib/python3.6/site-packages/cv2/data/haarcascade_frontalface_default.xml"

def return_bbx(image):
    faceCascade = cv2.CascadeClassifier(cascPath)
    faces = faceCascade.detectMultiScale(image, scaleFactor=1.1, minNeighbors=5, flags=cv2.CASCADE_SCALE_IMAGE)
    return faces

In [3]:
Dataset_path = '/Users/abdulrehman/Desktop/SML Project/FacesInTheWild/'

Celebs = pd.read_csv(Dataset_path+'lfw_allnames.csv')
Celebs = Celebs[Celebs['images']>10]
Celebs

,name,images
20,Abdullah_Gul,19
52,Adrien_Brody,12
127,Alejandro_Toledo,39
210,Alvaro_Uribe,35
223,Amelie_Mauresmo,21
...,...,...
5541,Vicente_Fox,32
5569,Vladimir_Putin,49
5605,Wen_Jiabao,13
5659,Winona_Ryder,24


In [4]:
cat_list = []
X = []
Y = []
y_label = 0

for _, [name,__] in Celebs.iterrows():
    celeb_path = Dataset_path+'lfw-deepfunneled/'+name+'/'
    
    images_paths = get_files(celeb_path)
    temp = []
    for image_path in images_paths:
        image = cv2.imread(celeb_path+image_path,1)
        faces = return_bbx(image)
        if len(faces) == 1:
            if len(temp)>=10:
                break
            temp.append(len(X))
            (x,y,w,h) = faces[0]
            cropped = image[x:x+w, y:y+h]
            dim = (100, 100)
            resized = cv2.resize(cropped, dim, interpolation = cv2.INTER_AREA)
            image = np.array(resized).astype("float32")
            X.append(image)
            Y.append(y_label)
    y_label+=1
    cat_list.append(temp)
        



In [5]:
X_data = np.asarray(X)/255
Y_data = np.array(Y)
cat_list = np.asarray(cat_list)

print(X_data.shape, Y_data.shape, cat_list.shape)

(1423, 100, 100, 3) (1423,) (143,)


In [6]:
from collections import Counter

Counter(Y_data)

Counter({0: 10,
         1: 10,
         2: 10,
         3: 10,
         4: 10,
         5: 10,
         6: 10,
         7: 10,
         8: 10,
         9: 10,
         10: 10,
         11: 10,
         12: 10,
         13: 10,
         14: 10,
         15: 10,
         16: 10,
         17: 10,
         18: 10,
         19: 10,
         20: 10,
         21: 10,
         22: 10,
         23: 10,
         24: 10,
         25: 10,
         26: 10,
         27: 10,
         28: 10,
         29: 10,
         30: 10,
         31: 10,
         32: 10,
         33: 10,
         34: 10,
         35: 10,
         36: 10,
         37: 9,
         38: 10,
         39: 10,
         40: 10,
         41: 10,
         42: 10,
         43: 10,
         44: 10,
         45: 10,
         46: 10,
         47: 10,
         48: 10,
         49: 10,
         50: 10,
         51: 10,
         52: 10,
         53: 10,
         54: 10,
         55: 10,
         56: 10,
         57: 10,
         58: 10,
        

In [7]:

a = Y_data
n_classes = len(set(a))
n_classes

143

In [8]:
train_split = 0.9

train_size = int(n_classes*train_split)
test_size = n_classes-train_size

train_files = train_size * 10

X_train = X_data[:train_files]
y_train = Y_data[:train_files]
cat_train = cat_list[:train_size]

#Validation Split
X_test = X_data[train_files:]
y_test = Y_data[train_files:]
cat_test = cat_list[train_size:]

print('X&Y shape of training data :',X_train.shape, 'and', y_train.shape, cat_train.shape)
print('X&Y shape of testing data :' , X_test.shape, 'and', y_test.shape, cat_test.shape)

X&Y shape of training data : (1280, 100, 100, 3) and (1280,) (128,)
X&Y shape of testing data : (143, 100, 100, 3) and (143,) (15,)


In [9]:
def get_batch(batch_size=64):
    
    temp_x = X_train
    temp_cat_list = cat_train
    start=0
    end=train_size
    batch_x=[]
        
    batch_y = np.zeros(batch_size)
    batch_y[int(batch_size/2):] = 1
    np.random.shuffle(batch_y)
    
    class_list = np.random.randint(start, end, batch_size) 
    batch_x.append(np.zeros((batch_size, 100, 100, 3)))
    batch_x.append(np.zeros((batch_size, 100, 100, 3)))

    for i in range(0, batch_size):
        batch_x[0][i] = temp_x[np.random.choice(temp_cat_list[class_list[i]])]  
        #If train_y has 0 pick from the same class, else pick from any other class
        if batch_y[i]==0:
            batch_x[1][i] = temp_x[np.random.choice(temp_cat_list[class_list[i]])]

        else:
            temp_list = np.append(temp_cat_list[:class_list[i]].flatten(), temp_cat_list[class_list[i]+1:].flatten())
            temp_list = np.random.choice(temp_list)
            batch_x[1][i] = temp_x[np.random.choice(temp_list)]
            
            
    return(batch_x, batch_y)

In [16]:
from keras.applications import VGG16
from keras.layers import Dense, Dropout, Activation, Flatten, Subtract, Input, BatchNormalization
from keras.models import Model, Sequential
from keras.optimizers import Adam
from keras.regularizers import l2
from keras.utils import plot_model
# from keras.layers.advanced_activations import LeakyReLU

input_shape = (100,100,3)

vgg_left = VGG16(weights = 'imagenet',include_top = False, input_shape = input_shape)

for layer in vgg_left.layers:
    layer.trainable = False
    layer._name = 'left_'+layer.name
    
left = [layer.output for layer in vgg_left.layers][-5]

left = Flatten()(left)
# left = Dropout(0.5)(left)
left = Dense(4096, kernel_regularizer=l2(1e-2))(left)
left = BatchNormalization()(left)
left = Activation('sigmoid')(left)


vgg_right = VGG16(weights = 'imagenet',include_top = False, input_shape = input_shape)

for layer in vgg_right.layers:
    layer.trainable = False
    layer._name = 'right_'+layer.name

right = [layer.output for layer in vgg_right.layers][-5]

right = Flatten()(right)
# right = Dropout(0.5)(right)
right = Dense(4096, kernel_regularizer=l2(1e-2))(right)
right = BatchNormalization()(right)
right = Activation('sigmoid')(right)


subtracted = Subtract()([left,right])
# subtracted = Dense(128, activation='sigmoid')(subtracted)
out = Dense(1, activation='sigmoid')(subtracted)

model = Model(inputs = [vgg_left.input,vgg_right.input], outputs = out)

model.compile(loss="binary_crossentropy", optimizer=Adam(0.0001), metrics=['accuracy'])

model.summary()

Model: "functional_7"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
left_input_7 (InputLayer)       [(None, 100, 100, 3) 0                                            
__________________________________________________________________________________________________
right_input_8 (InputLayer)      [(None, 100, 100, 3) 0                                            
__________________________________________________________________________________________________
left_block1_conv1 (Conv2D)      (None, 100, 100, 64) 1792        left_input_7[0][0]               
__________________________________________________________________________________________________
right_block1_conv1 (Conv2D)     (None, 100, 100, 64) 1792        right_input_8[0][0]              
_______________________________________________________________________________________

In [11]:
# from keras.applications import VGG16
# from keras.layers import Input, Dense, Flatten, Subtract, Conv2D, MaxPooling2D
# from keras.layers import Dropout, Activation, BatchNormalization
# from keras.models import Model, Sequential
# from keras.optimizers import Adam
# from keras.regularizers import l2
# from keras.utils import plot_model
# # from keras.layers.advanced_activations import LeakyReLU

# input_shape = (100,100,3)

# left_input = Input(input_shape)
# right_input = Input(input_shape)

# left = Sequential()
# left.add(left_input)
# left.add(Conv2D(64, (3,3), activation='relu'))
# left.add(MaxPooling2D(2,2))
# left.add(Conv2D(128, (3,3), activation='relu'))
# left.add(MaxPooling2D(2,2))
# left.add(Conv2D(128, (3,3), activation='relu'))
# left.add(MaxPooling2D(2,2))
# left.add(Conv2D(256, (3,3), activation='relu'))
# left.add(MaxPooling2D(2,2))
# left.add(Flatten())
# left.add(Dense(1028, activation='relu', kernel_regularizer=l2(1e-2)))

# right = Sequential()
# right.add(right_input)
# right.add(Conv2D(64, (3,3), activation='relu'))
# right.add(MaxPooling2D(2,2))
# right.add(Conv2D(128, (3,3), activation='relu'))
# right.add(MaxPooling2D(2,2))
# right.add(Conv2D(128, (3,3), activation='relu'))
# right.add(MaxPooling2D(2,2))
# right.add(Conv2D(256, (3,3), activation='relu'))
# right.add(MaxPooling2D(2,2))
# right.add(Flatten())
# right.add(Dense(1028, activation='relu', kernel_regularizer=l2(1e-2)))

# subtracted = Subtract()([left.output,right.output])
# subtracted = Dense(128, activation='sigmoid')(subtracted)
# out = Dense(1, activation='sigmoid')(subtracted)

# model = Model(inputs = [left.input, right.input], outputs = out)

# model.compile(loss="binary_crossentropy", optimizer=Adam(0.0001), metrics=['accuracy'])

# model.summary()

In [12]:
def one_shot_learning(model, n_way, n_val):
    
    temp_x = X_test
    temp_cat_list = cat_test
    batch_x=[]
    x_0_choice=[]
    n_correct = 0
   
    class_list = np.random.randint(train_size+1, n_classes-1, n_val)

    for i in class_list:  
        j = np.random.choice(cat_list[i])
        temp=[]
        temp.append(np.zeros((n_way, 100, 100, 3)))
        temp.append(np.zeros((n_way, 100, 100, 3)))
        for k in range(0, n_way):
            temp[0][k] = X_data[j]
            
            if k==0:
                temp[1][k] = X_data[np.random.choice(cat_list[i])]
            else:
                temp_list = np.append(cat_list[:i].flatten(), cat_list[i+1:].flatten())
                temp_list = np.random.choice(temp_list)
                temp[1][k] = X_data[np.random.choice(temp_list)]

        result = model.predict(temp)
        result = result.flatten().tolist()
        result_index = result.index(min(result))
        if result_index == 0:
            n_correct = n_correct + 1
    print(n_correct, "correctly classified among", n_val)
    accuracy = (n_correct*100)/n_val
    return accuracy

[]

In [18]:
epochs = 10000
n_way = 20
n_val = 64
batch_size = 64

loss_list=[]
accuracy_list=[]
for epoch in range(1,epochs):
    batch_x, batch_y = get_batch(batch_size)
    loss = model.train_on_batch(batch_x, batch_y)
    loss_list.append((epoch,loss))
    print('Epoch:', epoch, ', Loss:',loss)
    if epoch%250 == 0:
        print("=============================================")
        accuracy = one_shot_learning(model, n_way, n_val)
        accuracy_list.append((epoch, accuracy))
        print('Accuracy as of', epoch, 'epochs:', accuracy)
        print("=============================================")
        if(accuracy>99):
            print("Achieved more than 90% Accuracy")

Epoch: 1 , Loss: [134.71145629882812, 0.578125]
Epoch: 2 , Loss: [132.87075805664062, 0.5]
Epoch: 3 , Loss: [131.03445434570312, 0.484375]
Epoch: 4 , Loss: [129.51890563964844, 0.375]
Epoch: 5 , Loss: [127.78551483154297, 0.453125]
Epoch: 6 , Loss: [125.80372619628906, 0.46875]
Epoch: 7 , Loss: [123.91171264648438, 0.515625]
Epoch: 8 , Loss: [122.10346984863281, 0.578125]
Epoch: 9 , Loss: [120.44327545166016, 0.546875]
Epoch: 10 , Loss: [119.02418518066406, 0.390625]
Epoch: 11 , Loss: [117.15896606445312, 0.5625]
Epoch: 12 , Loss: [115.63201904296875, 0.421875]
Epoch: 13 , Loss: [113.78555297851562, 0.484375]
Epoch: 14 , Loss: [112.2431869506836, 0.515625]
Epoch: 15 , Loss: [110.67958068847656, 0.53125]
Epoch: 16 , Loss: [108.74568176269531, 0.484375]
Epoch: 17 , Loss: [107.40313720703125, 0.53125]
Epoch: 18 , Loss: [105.84418487548828, 0.453125]
Epoch: 19 , Loss: [105.04824829101562, 0.390625]
Epoch: 20 , Loss: [103.12879180908203, 0.46875]
Epoch: 21 , Loss: [101.14493560791016, 0.609

Epoch: 174 , Loss: [12.617393493652344, 0.546875]
Epoch: 175 , Loss: [12.500940322875977, 0.46875]
Epoch: 176 , Loss: [12.308443069458008, 0.53125]
Epoch: 177 , Loss: [12.117708206176758, 0.578125]
Epoch: 178 , Loss: [12.043132781982422, 0.53125]
Epoch: 179 , Loss: [11.943619728088379, 0.5]
Epoch: 180 , Loss: [11.882610321044922, 0.515625]
Epoch: 181 , Loss: [11.633068084716797, 0.515625]
Epoch: 182 , Loss: [11.589685440063477, 0.46875]
Epoch: 183 , Loss: [11.479053497314453, 0.453125]
Epoch: 184 , Loss: [11.204776763916016, 0.5625]
Epoch: 185 , Loss: [11.063711166381836, 0.5625]
Epoch: 186 , Loss: [10.819282531738281, 0.625]
Epoch: 187 , Loss: [10.7879638671875, 0.484375]
Epoch: 188 , Loss: [10.78528118133545, 0.375]
Epoch: 189 , Loss: [10.550376892089844, 0.484375]
Epoch: 190 , Loss: [10.54350757598877, 0.421875]
Epoch: 191 , Loss: [10.351387023925781, 0.5]
Epoch: 192 , Loss: [10.321025848388672, 0.46875]
Epoch: 193 , Loss: [10.005826950073242, 0.546875]
Epoch: 194 , Loss: [10.175245

Epoch: 341 , Loss: [2.8870673179626465, 0.453125]
Epoch: 342 , Loss: [2.894804000854492, 0.453125]
Epoch: 343 , Loss: [2.8365442752838135, 0.46875]
Epoch: 344 , Loss: [2.739819049835205, 0.546875]
Epoch: 345 , Loss: [2.7787344455718994, 0.46875]
Epoch: 346 , Loss: [2.9192092418670654, 0.375]
Epoch: 347 , Loss: [2.713193893432617, 0.578125]
Epoch: 348 , Loss: [2.752960681915283, 0.5625]
Epoch: 349 , Loss: [2.7465946674346924, 0.453125]
Epoch: 350 , Loss: [2.5354130268096924, 0.625]
Epoch: 351 , Loss: [2.639453887939453, 0.515625]
Epoch: 352 , Loss: [2.6730494499206543, 0.4375]
Epoch: 353 , Loss: [2.690403938293457, 0.46875]
Epoch: 354 , Loss: [2.770460844039917, 0.40625]
Epoch: 355 , Loss: [2.585625410079956, 0.546875]
Epoch: 356 , Loss: [2.5912013053894043, 0.515625]
Epoch: 357 , Loss: [2.583632230758667, 0.4375]
Epoch: 358 , Loss: [2.551177501678467, 0.46875]
Epoch: 359 , Loss: [2.6451148986816406, 0.46875]
Epoch: 360 , Loss: [2.5640742778778076, 0.40625]
Epoch: 361 , Loss: [2.6204590

Epoch: 506 , Loss: [1.3402433395385742, 0.515625]
Epoch: 507 , Loss: [1.3389471769332886, 0.421875]
Epoch: 508 , Loss: [1.3057879209518433, 0.5]
Epoch: 509 , Loss: [1.3975839614868164, 0.4375]
Epoch: 510 , Loss: [1.29679274559021, 0.578125]
Epoch: 511 , Loss: [1.2843254804611206, 0.53125]
Epoch: 512 , Loss: [1.275181531906128, 0.546875]
Epoch: 513 , Loss: [1.311950922012329, 0.515625]
Epoch: 514 , Loss: [1.2736895084381104, 0.515625]
Epoch: 515 , Loss: [1.2668123245239258, 0.59375]
Epoch: 516 , Loss: [1.3295429944992065, 0.46875]
Epoch: 517 , Loss: [1.3528109788894653, 0.421875]
Epoch: 518 , Loss: [1.2747678756713867, 0.53125]
Epoch: 519 , Loss: [1.220332384109497, 0.59375]
Epoch: 520 , Loss: [1.2520220279693604, 0.59375]
Epoch: 521 , Loss: [1.2588272094726562, 0.546875]
Epoch: 522 , Loss: [1.2681409120559692, 0.5625]
Epoch: 523 , Loss: [1.282322883605957, 0.484375]
Epoch: 524 , Loss: [1.2822538614273071, 0.453125]
Epoch: 525 , Loss: [1.2370988130569458, 0.53125]
Epoch: 526 , Loss: [1.

Epoch: 675 , Loss: [0.9948355555534363, 0.453125]
Epoch: 676 , Loss: [1.0486313104629517, 0.46875]
Epoch: 677 , Loss: [0.9586559534072876, 0.515625]
Epoch: 678 , Loss: [0.9955691695213318, 0.515625]
Epoch: 679 , Loss: [0.9542880654335022, 0.578125]
Epoch: 680 , Loss: [0.9854270219802856, 0.484375]
Epoch: 681 , Loss: [0.9558144807815552, 0.453125]
Epoch: 682 , Loss: [0.9905003905296326, 0.53125]
Epoch: 683 , Loss: [0.9385541677474976, 0.546875]
Epoch: 684 , Loss: [0.9595223069190979, 0.515625]
Epoch: 685 , Loss: [1.0013117790222168, 0.484375]
Epoch: 686 , Loss: [0.9227609634399414, 0.578125]
Epoch: 687 , Loss: [0.9269557595252991, 0.53125]
Epoch: 688 , Loss: [0.928653359413147, 0.53125]
Epoch: 689 , Loss: [0.9856292009353638, 0.453125]
Epoch: 690 , Loss: [0.9418770670890808, 0.5625]
Epoch: 691 , Loss: [0.9918065071105957, 0.4375]
Epoch: 692 , Loss: [0.8612917065620422, 0.578125]
Epoch: 693 , Loss: [0.9926767349243164, 0.4375]
Epoch: 694 , Loss: [0.9582979679107666, 0.40625]
Epoch: 695 ,

KeyboardInterrupt: 